In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import stats as sps
import dcor
from dcor.independence import distance_covariance_test
from joblib import Parallel, delayed
import numpy as np
from scipy import stats as sps
from scipy.stats import pearsonr

In [2]:
df = pd.read_csv('nba_game_data_updated.csv')
grouped_games = df.groupby('game_id')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
pd.set_option('display.max_columns', None)
df = df.loc[:, ~df.columns.str.contains('_opp')]
df.head(4)

,FG_basic_totals,FGA_basic_totals,FG%_basic_totals,3P_basic_totals,3PA_basic_totals,3P%_basic_totals,FT_basic_totals,FTA_basic_totals,FT%_basic_totals,ORB_basic_totals,DRB_basic_totals,TRB_basic_totals,AST_basic_totals,STL_basic_totals,BLK_basic_totals,TOV_basic_totals,PF_basic_totals,PTS_basic_totals,MP_advanced_totals,TS%_advanced_totals,eFG%_advanced_totals,3PAr_advanced_totals,FTr_advanced_totals,ORB%_advanced_totals,DRB%_advanced_totals,TRB%_advanced_totals,AST%_advanced_totals,STL%_advanced_totals,BLK%_advanced_totals,TOV%_advanced_totals,USG%_advanced_totals,ORtg_advanced_totals,DRtg_advanced_totals,FG_basic_max,FGA_basic_max,FG%_basic_max,3P_basic_max,3PA_basic_max,3P%_basic_max,FT_basic_max,FTA_basic_max,FT%_basic_max,ORB_basic_max,DRB_basic_max,TRB_basic_max,AST_basic_max,STL_basic_max,BLK_basic_max,TOV_basic_max,PF_basic_max,PTS_basic_max,GmSc_basic_max,+/-_basic_max,TS%_advanced_max,eFG%_advanced_max,3PAr_advanced_max,FTr_advanced_max,ORB%_advanced_max,DRB%_advanced_max,TRB%_advanced_max,AST%_advanced_max,STL%_advanced_max,BLK%_advanced_max,TOV%_advanced_max,USG%_advanced_max,ORtg_advanced_max,DRtg_advanced_max,Team,Q1,Q2,Q3,Q4,Total,home,season,date,OT5,OT6,OT7,OT8,game_id,win
0,32.0,72.0,0.444,9.0,19.0,0.474,22.0,26.0,0.846,6.0,32.0,38.0,20.0,9.0,8.0,16.0,27.0,95.0,240.0,0.569,0.507,0.264,0.361,18.8,84.2,54.3,62.5,10.0,15.1,16.1,100.0,106.0,99.3,6.0,16.0,0.600,3.0,6.0,1.000,9.0,10.0,1.000,3.0,10.0,11.0,10.0,3.0,3.0,3.0,5.0,23.0,18.0,11.0,0.725,0.700,1.000,4.0,13.6,33.3,20.7,52.7,4.7,8.2,28.7,25.2,125.0,107.0,BOS,21,30,21,23,95,0,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_BOS_CLE,1
1,29.0,70.0,0.414,6.0,17.0,0.353,25.0,32.0,0.781,6.0,26.0,32.0,17.0,7.0,9.0,14.0,22.0,89.0,240.0,0.529,0.457,0.243,0.457,15.8,81.3,45.7,58.6,7.9,17.0,14.3,100.0,99.9,106.6,12.0,22.0,0.545,4.0,9.0,0.667,10.0,13.0,1.000,5.0,10.0,10.0,8.0,2.0,4.0,5.0,5.0,38.0,31.2,3.0,0.685,0.636,0.409,1.0,19.2,52.1,23.8,52.9,8.7,8.1,25.0,36.0,121.0,115.0,CLE,28,17,20,24,89,1,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_BOS_CLE,0
2,39.0,84.0,0.464,4.0,13.0,0.308,20.0,24.0,0.833,9.0,37.0,46.0,19.0,6.0,4.0,10.0,29.0,102.0,240.0,0.539,0.488,0.155,0.286,22.5,77.1,52.3,48.7,6.7,6.9,9.6,100.0,113.3,101.1,10.0,21.0,1.000,2.0,4.0,0.667,8.0,9.0,1.000,3.0,9.0,10.0,9.0,2.0,2.0,3.0,5.0,29.0,20.6,16.0,1.025,1.000,0.750,1.0,10.3,36.0,19.6,42.6,4.8,4.7,45.0,33.8,158.0,112.0,WAS,21,35,21,25,102,0,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_DAL_WAS,1
3,30.0,76.0,0.395,4.0,18.0,0.222,27.0,34.0,0.794,11.0,31.0,42.0,16.0,6.0,9.0,10.0,20.0,91.0,240.0,0.500,0.421,0.237,0.447,22.9,77.5,47.7,53.3,6.7,12.7,9.9,100.0,101.1,113.3,10.0,25.0,1.000,2.0,6.0,0.500,12.0,13.0,0.923,4.0,8.0,9.0,6.0,2.0,3.0,3.0,4.0,34.0,21.5,7.0,0.862,1.000,0.500,3.0,12.8,30.7,14.0,37.5,6.9,8.7,25.0,41.6,213.0,123.0,DAL,21,29,24,17,91,1,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_DAL_WAS,0


In [3]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["date"])
df.head(4)
home_df = df[df['home'] == 1].copy()
home_game_ids = home_df['game_id']
home_df = home_df.add_prefix('home_')
home_df['game_id'] = home_game_ids.values
away_df = df[df['home'] == 0].copy()
away_game_ids = away_df['game_id']
away_df = away_df.add_prefix('away_')
away_df['game_id'] = away_game_ids.values
df = pd.merge(home_df, away_df, on='game_id')
df = df.drop(columns=['home_home', 'away_home', 'home_game_id', 'away_season', 'away_date', 'away_game_id', 'away_win'])
df = df.rename(columns={'home_season': 'season', 'home_date': 'date'})
df.head(4)

,home_FG_basic_totals,home_FGA_basic_totals,home_FG%_basic_totals,home_3P_basic_totals,home_3PA_basic_totals,home_3P%_basic_totals,home_FT_basic_totals,home_FTA_basic_totals,home_FT%_basic_totals,home_ORB_basic_totals,home_DRB_basic_totals,home_TRB_basic_totals,home_AST_basic_totals,home_STL_basic_totals,home_BLK_basic_totals,home_TOV_basic_totals,home_PF_basic_totals,home_PTS_basic_totals,home_MP_advanced_totals,home_TS%_advanced_totals,home_eFG%_advanced_totals,home_3PAr_advanced_totals,home_FTr_advanced_totals,home_ORB%_advanced_totals,home_DRB%_advanced_totals,home_TRB%_advanced_totals,home_AST%_advanced_totals,home_STL%_advanced_totals,home_BLK%_advanced_totals,home_TOV%_advanced_totals,home_USG%_advanced_totals,home_ORtg_advanced_totals,home_DRtg_advanced_totals,home_FG_basic_max,home_FGA_basic_max,home_FG%_basic_max,home_3P_basic_max,home_3PA_basic_max,home_3P%_basic_max,home_FT_basic_max,home_FTA_basic_max,home_FT%_basic_max,home_ORB_basic_max,home_DRB_basic_max,home_TRB_basic_max,home_AST_basic_max,home_STL_basic_max,home_BLK_basic_max,home_TOV_basic_max,home_PF_basic_max,home_PTS_basic_max,home_GmSc_basic_max,home_+/-_basic_max,home_TS%_advanced_max,home_eFG%_advanced_max,home_3PAr_advanced_max,home_FTr_advanced_max,home_ORB%_advanced_max,home_DRB%_advanced_max,home_TRB%_advanced_max,home_AST%_advanced_max,home_STL%_advanced_max,home_BLK%_advanced_max,home_TOV%_advanced_max,home_USG%_advanced_max,home_ORtg_advanced_max,home_DRtg_advanced_max,home_Team,home_Q1,home_Q2,home_Q3,home_Q4,home_Total,season,date,home_OT5,home_OT6,home_OT7,home_OT8,home_win,game_id,away_FG_basic_totals,away_FGA_basic_totals,away_FG%_basic_totals,away_3P_basic_totals,away_3PA_basic_totals,away_3P%_basic_totals,away_FT_basic_totals,away_FTA_basic_totals,away_FT%_basic_totals,away_ORB_basic_totals,away_DRB_basic_totals,away_TRB_basic_totals,away_AST_basic_totals,away_STL_basic_totals,away_BLK_basic_totals,away_TOV_basic_totals,away_PF_basic_totals,away_PTS_basic_totals,away_MP_advanced_totals,away_TS%_advanced_totals,away_eFG%_advanced_totals,away_3PAr_advanced_totals,away_FTr_advanced_totals,away_ORB%_advanced_totals,away_DRB%_advanced_totals,away_TRB%_advanced_totals,away_AST%_advanced_totals,away_STL%_advanced_totals,away_BLK%_advanced_totals,away_TOV%_advanced_totals,away_USG%_advanced_totals,away_ORtg_advanced_totals,away_DRtg_advanced_totals,away_FG_basic_max,away_FGA_basic_max,away_FG%_basic_max,away_3P_basic_max,away_3PA_basic_max,away_3P%_basic_max,away_FT_basic_max,away_FTA_basic_max,away_FT%_basic_max,away_ORB_basic_max,away_DRB_basic_max,away_TRB_basic_max,away_AST_basic_max,away_STL_basic_max,away_BLK_basic_max,away_TOV_basic_max,away_PF_basic_max,away_PTS_basic_max,away_GmSc_basic_max,away_+/-_basic_max,away_TS%_advanced_max,away_eFG%_advanced_max,away_3PAr_advanced_max,away_FTr_advanced_max,away_ORB%_advanced_max,away_DRB%_advanced_max,away_TRB%_advanced_max,away_AST%_advanced_max,away_STL%_advanced_max,away_BLK%_advanced_max,away_TOV%_advanced_max,away_USG%_advanced_max,away_ORtg_advanced_max,away_DRtg_advanced_max,away_Team,away_Q1,away_Q2,away_Q3,away_Q4,away_Total,away_OT5,away_OT6,away_OT7,away_OT8
0,29.0,70.0,0.414,6.0,17.0,0.353,25.0,32.0,0.781,6.0,26.0,32.0,17.0,7.0,9.0,14.0,22.0,89.0,240.0,0.529,0.457,0.243,0.457,15.8,81.3,45.7,58.6,7.9,17.0,14.3,100.0,99.9,106.6,12.0,22.0,0.545,4.0,9.0,0.667,10.0,13.0,1.000,5.0,10.0,10.0,8.0,2.0,4.0,5.0,5.0,38.0,31.2,3.0,0.685,0.636,0.409,1.000,19.2,52.1,23.8,52.9,8.7,8.1,25.0,36.0,121.0,115.0,CLE,28,17,20,24,89,2010,2009-10-27,0.0,0.0,0.0,0.0,0,2009-10-27_BOS_CLE,32.0,72.0,0.444,9.0,19.0,0.474,22.0,26.0,0.846,6.0,32.0,38.0,20.0,9.0,8.0,16.0,27.0,95.0,240.0,0.569,0.507,0.264,0.361,18.8,84.2,54.3,62.5,10.0,15.1,16.1,100.0,106.0,99.3,6.0,16.0,0.6,3.0,6.0,1.000,9.0,10.0,1.0,3.0,10.0,11.0,10.0,3.0,3.0,3.0,5.0,23.0,18.0,11.0,0.725,0.700,1.00,4.000,13.6,33.3,20.7,52.7,4.7,8.2,28.7,25.2,125.0,107.0,BOS,21,30,21,23,95,0.0,0.0,0.0,0.0
1,30.0,76.0,0.395,4.0,18.0,0.222,27.0,34.0,0.794,11.0,31.0,42

In [4]:
df["Team_A"] = df.apply(lambda row: min(row["home_Team"], row["away_Team"]), axis=1)
df["Team_B"] = df.apply(lambda row: max(row["home_Team"], row["away_Team"]), axis=1)
df["matchup_id"] = df["Team_A"] + "_vs_" + df["Team_B"]
df.head(1)

,home_FG_basic_totals,home_FGA_basic_totals,home_FG%_basic_totals,home_3P_basic_totals,home_3PA_basic_totals,home_3P%_basic_totals,home_FT_basic_totals,home_FTA_basic_totals,home_FT%_basic_totals,home_ORB_basic_totals,home_DRB_basic_totals,home_TRB_basic_totals,home_AST_basic_totals,home_STL_basic_totals,home_BLK_basic_totals,home_TOV_basic_totals,home_PF_basic_totals,home_PTS_basic_totals,home_MP_advanced_totals,home_TS%_advanced_totals,home_eFG%_advanced_totals,home_3PAr_advanced_totals,home_FTr_advanced_totals,home_ORB%_advanced_totals,home_DRB%_advanced_totals,home_TRB%_advanced_totals,home_AST%_advanced_totals,home_STL%_advanced_totals,home_BLK%_advanced_totals,home_TOV%_advanced_totals,home_USG%_advanced_totals,home_ORtg_advanced_totals,home_DRtg_advanced_totals,home_FG_basic_max,home_FGA_basic_max,home_FG%_basic_max,home_3P_basic_max,home_3PA_basic_max,home_3P%_basic_max,home_FT_basic_max,home_FTA_basic_max,home_FT%_basic_max,home_ORB_basic_max,home_DRB_basic_max,home_TRB_basic_max,home_AST_basic_max,home_STL_basic_max,home_BLK_basic_max,home_TOV_basic_max,home_PF_basic_max,home_PTS_basic_max,home_GmSc_basic_max,home_+/-_basic_max,home_TS%_advanced_max,home_eFG%_advanced_max,home_3PAr_advanced_max,home_FTr_advanced_max,home_ORB%_advanced_max,home_DRB%_advanced_max,home_TRB%_advanced_max,home_AST%_advanced_max,home_STL%_advanced_max,home_BLK%_advanced_max,home_TOV%_advanced_max,home_USG%_advanced_max,home_ORtg_advanced_max,home_DRtg_advanced_max,home_Team,home_Q1,home_Q2,home_Q3,home_Q4,home_Total,season,date,home_OT5,home_OT6,home_OT7,home_OT8,home_win,game_id,away_FG_basic_totals,away_FGA_basic_totals,away_FG%_basic_totals,away_3P_basic_totals,away_3PA_basic_totals,away_3P%_basic_totals,away_FT_basic_totals,away_FTA_basic_totals,away_FT%_basic_totals,away_ORB_basic_totals,away_DRB_basic_totals,away_TRB_basic_totals,away_AST_basic_totals,away_STL_basic_totals,away_BLK_basic_totals,away_TOV_basic_totals,away_PF_basic_totals,away_PTS_basic_totals,away_MP_advanced_totals,away_TS%_advanced_totals,away_eFG%_advanced_totals,away_3PAr_advanced_totals,away_FTr_advanced_totals,away_ORB%_advanced_totals,away_DRB%_advanced_totals,away_TRB%_advanced_totals,away_AST%_advanced_totals,away_STL%_advanced_totals,away_BLK%_advanced_totals,away_TOV%_advanced_totals,away_USG%_advanced_totals,away_ORtg_advanced_totals,away_DRtg_advanced_totals,away_FG_basic_max,away_FGA_basic_max,away_FG%_basic_max,away_3P_basic_max,away_3PA_basic_max,away_3P%_basic_max,away_FT_basic_max,away_FTA_basic_max,away_FT%_basic_max,away_ORB_basic_max,away_DRB_basic_max,away_TRB_basic_max,away_AST_basic_max,away_STL_basic_max,away_BLK_basic_max,away_TOV_basic_max,away_PF_basic_max,away_PTS_basic_max,away_GmSc_basic_max,away_+/-_basic_max,away_TS%_advanced_max,away_eFG%_advanced_max,away_3PAr_advanced_max,away_FTr_advanced_max,away_ORB%_advanced_max,away_DRB%_advanced_max,away_TRB%_advanced_max,away_AST%_advanced_max,away_STL%_advanced_max,away_BLK%_advanced_max,away_TOV%_advanced_max,away_USG%_advanced_max,away_ORtg_advanced_max,away_DRtg_advanced_max,away_Team,away_Q1,away_Q2,away_Q3,away_Q4,away_Total,away_OT5,away_OT6,away_OT7,away_OT8,Team_A,Team_B,matchup_id
0,29.0,70.0,0.414,6.0,17.0,0.353,25.0,32.0,0.781,6.0,26.0,32.0,17.0,7.0,9.0,14.0,22.0,89.0,240.0,0.529,0.457,0.243,0.457,15.8,81.3,45.7,58.6,7.9,17.0,14.3,100.0,99.9,106.6,12.0,22.0,0.545,4.0,9.0,0.667,10.0,13.0,1.0,5.0,10.0,10.0,8.0,2.0,4.0,5.0,5.0,38.0,31.2,3.0,0.685,0.636,0.409,1.0,19.2,52.1,23.8,52.9,8.7,8.1,25.0,36.0,121.0,115.0,CLE,28,17,20,24,89,2010,2009-10-27,0.0,0.0,0.0,0.0,0,2009-10-27_BOS_CLE,32.0,72.0,0.444,9.0,19.0,0.474,22.0,26.0,0.846,6.0,32.0,38.0,20.0,9.0,8.0,16.0,27.0,95.0,240.0,0.569,0.507,0.264,0.361,18.8,84.2,54.3,62.5,10.0,15.1,16.1,100.0,106.0,99.3,6.0,16.0,0.6,3.0,6.0,1.0,9.0,10.0,1.0,3.0,10.0,11.0,10.0,3.0,3.0,3.0,5.0,23.0,18.0,11.0,0.725,0.7,1.0,4.0,13.6,33.3,20.7,52.7,4.7,8.2,28.7,25.2,125.0,107.0,BOS,21,30,21,23,95,0.0,0.0,0.0,0.0,BOS,CLE,BOS_vs_CLE


In [ ]:
def assign_team_stats(row):
    if row['home_Team'] == row['Team_A']:
        team_a_stats = row.filter(like='home_').rename(lambda x: x.replace('home_', 'Team_A_'))
        team_b_stats = row.filter(like='away_').rename(lambda x: x.replace('away_', 'Team_B_'))
    else:
        team_a_stats = row.filter(like='away_').rename(lambda x: x.replace('away_', 'Team_A_'))
        team_b_stats = row.filter(like='home_').rename(lambda x: x.replace('home_', 'Team_B_'))
    return pd.concat([team_a_stats, team_b_stats])
team_stats_df = df.apply(assign_team_stats, axis=1)
df = pd.concat([df[['Team_A', 'Team_B', 'matchup_id', 'date', 'season', 'home_Team']], team_stats_df], axis=1)


In [ ]:
df = df.drop(columns=['Team_A_win', 'Team_B_win'])
df['Team_A_win'] = df.apply(lambda row: 1 if row['Team_A_PTS_basic_totals'] > row['Team_B_PTS_basic_totals'] else 0, axis=1)
df.head(5)

In [ ]:
df = df.copy()
df['Team_A_home'] = df.apply(lambda row: 1 if row['home_Team'] == row['Team_A'] else 0, axis=1)
df=df.drop(columns=['home_Team', 'Team_A_Team', 'Team_B_Team'])
df.head(2)

In [ ]:
df = df.sort_values(by=['matchup_id', 'date']).reset_index(drop=True)
df['Team_A_win_next'] = df.groupby('matchup_id')['Team_A_win'].shift(-1)
df.head(6)

In [ ]:
playoff_start_dates = {
    2010: '2010-04-17',
    2011: '2011-04-16',
    2012: '2012-04-28',
    2013: '2013-04-20',
    2014: '2014-04-19',
    2015: '2015-04-18',
    2016: '2016-04-16',
    2017: '2017-04-15',
    2018: '2018-04-14',
    2019: '2019-04-13',
    2020: '2020-08-17',
    2021: '2021-05-22',
    2022: '2022-04-16',
    2023: '2023-04-15',
    2024: '2024-04-20'
}
df['playoff_game'] = df.apply(lambda row: 1 if row['date'] >= pd.to_datetime(playoff_start_dates.get(row['season'])) else 0, axis=1)
df = df.drop(columns=['season'])
print(df['playoff_game'].value_counts())
print(df['Team_A_win_next'].value_counts())

In [ ]:
#df.head(10)
print(f'Total missing values: {df.isnull().sum().sum()}')
print(f"Total missing values in Team_win_next label: {df['Team_A_win_next'].isnull().sum()}")
df = df.dropna(axis=0, subset=['Team_A_win_next'])

In [ ]:
df.columns.tolist()

In [ ]:
num_games = df.shape[0]
print(f'Total games from the 2009-2010 season to the 2023-2024 season: {num_games}')
num_features = df.shape[1] - 1
print(f'Number of features: {num_features}')
categorical_variables = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f'Categorical variables: {categorical_variables}')

In [ ]:
numerical_features  = ['Team_A_+/-_basic_max',
 'Team_A_3P%_basic_max',
 'Team_A_3P%_basic_totals',
 'Team_A_3PA_basic_max',
 'Team_A_3PA_basic_totals',
 'Team_A_3PAr_advanced_max',
 'Team_A_3PAr_advanced_totals',
 'Team_A_3P_basic_max',
 'Team_A_3P_basic_totals',
 'Team_A_AST%_advanced_max',
 'Team_A_AST%_advanced_totals',
 'Team_A_AST_basic_max',
 'Team_A_AST_basic_totals',
 'Team_A_BLK%_advanced_max',
 'Team_A_BLK%_advanced_totals',
 'Team_A_BLK_basic_max',
 'Team_A_BLK_basic_totals',
 'Team_A_DRB%_advanced_max',
 'Team_A_DRB%_advanced_totals',
 'Team_A_DRB_basic_max',
 'Team_A_DRB_basic_totals',
 'Team_A_DRtg_advanced_max',
 'Team_A_DRtg_advanced_totals',
 'Team_A_FG%_basic_max',
 'Team_A_FG%_basic_totals',
 'Team_A_FGA_basic_max',
 'Team_A_FGA_basic_totals',
 'Team_A_FG_basic_max',
 'Team_A_FG_basic_totals',
 'Team_A_FT%_basic_max',
 'Team_A_FT%_basic_totals',
 'Team_A_FTA_basic_max',
 'Team_A_FTA_basic_totals',
 'Team_A_FT_basic_max',
 'Team_A_FT_basic_totals',
 'Team_A_FTr_advanced_max',
 'Team_A_FTr_advanced_totals',
 'Team_A_GmSc_basic_max',
 'Team_A_MP_advanced_totals',
 'Team_A_ORB%_advanced_max',
 'Team_A_ORB%_advanced_totals',
 'Team_A_ORB_basic_max',
 'Team_A_ORB_basic_totals',
 'Team_A_ORtg_advanced_max',
 'Team_A_ORtg_advanced_totals',
 'Team_A_OT5',
 'Team_A_OT6',
 'Team_A_OT7',
 'Team_A_OT8',
 'Team_A_PF_basic_max',
 'Team_A_PF_basic_totals',
 'Team_A_PTS_basic_max',
 'Team_A_PTS_basic_totals',
 'Team_A_Q1',
 'Team_A_Q2',
 'Team_A_Q3',
 'Team_A_Q4',
 'Team_A_STL%_advanced_max',
 'Team_A_STL%_advanced_totals',
 'Team_A_STL_basic_max',
 'Team_A_STL_basic_totals',
 'Team_A_TOV%_advanced_max',
 'Team_A_TOV%_advanced_totals',
 'Team_A_TOV_basic_max',
 'Team_A_TOV_basic_totals',
 'Team_A_TRB%_advanced_max',
 'Team_A_TRB%_advanced_totals',
 'Team_A_TRB_basic_max',
 'Team_A_TRB_basic_totals',
 'Team_A_TS%_advanced_max',
 'Team_A_TS%_advanced_totals',
 'Team_A_Total',
 'Team_A_USG%_advanced_max',
 'Team_A_USG%_advanced_totals',
 'Team_A_eFG%_advanced_max',
 'Team_A_eFG%_advanced_totals',
 'Team_B_+/-_basic_max',
 'Team_B_3P%_basic_max',
 'Team_B_3P%_basic_totals',
 'Team_B_3PA_basic_max',
 'Team_B_3PA_basic_totals',
 'Team_B_3PAr_advanced_max',
 'Team_B_3PAr_advanced_totals',
 'Team_B_3P_basic_max',
 'Team_B_3P_basic_totals',
 'Team_B_AST%_advanced_max',
 'Team_B_AST%_advanced_totals',
 'Team_B_AST_basic_max',
 'Team_B_AST_basic_totals',
 'Team_B_BLK%_advanced_max',
 'Team_B_BLK%_advanced_totals',
 'Team_B_BLK_basic_max',
 'Team_B_BLK_basic_totals',
 'Team_B_DRB%_advanced_max',
 'Team_B_DRB%_advanced_totals',
 'Team_B_DRB_basic_max',
 'Team_B_DRB_basic_totals',
 'Team_B_DRtg_advanced_max',
 'Team_B_DRtg_advanced_totals',
 'Team_B_FG%_basic_max',
 'Team_B_FG%_basic_totals',
 'Team_B_FGA_basic_max',
 'Team_B_FGA_basic_totals',
 'Team_B_FG_basic_max',
 'Team_B_FG_basic_totals',
 'Team_B_FT%_basic_max',
 'Team_B_FT%_basic_totals',
 'Team_B_FTA_basic_max',
 'Team_B_FTA_basic_totals',
 'Team_B_FT_basic_max',
 'Team_B_FT_basic_totals',
 'Team_B_FTr_advanced_max',
 'Team_B_FTr_advanced_totals',
 'Team_B_GmSc_basic_max',
 'Team_B_MP_advanced_totals',
 'Team_B_ORB%_advanced_max',
 'Team_B_ORB%_advanced_totals',
 'Team_B_ORB_basic_max',
 'Team_B_ORB_basic_totals',
 'Team_B_ORtg_advanced_max',
 'Team_B_ORtg_advanced_totals',
 'Team_B_OT5',
 'Team_B_OT6',
 'Team_B_OT7',
 'Team_B_OT8',
 'Team_B_PF_basic_max',
 'Team_B_PF_basic_totals',
 'Team_B_PTS_basic_max',
 'Team_B_PTS_basic_totals',
 'Team_B_Q1',
 'Team_B_Q2',
 'Team_B_Q3',
 'Team_B_Q4',
 'Team_B_STL%_advanced_max',
 'Team_B_STL%_advanced_totals',
 'Team_B_STL_basic_max',
 'Team_B_STL_basic_totals',
 'Team_B_TOV%_advanced_max',
 'Team_B_TOV%_advanced_totals',
 'Team_B_TOV_basic_max',
 'Team_B_TOV_basic_totals',
 'Team_B_TRB%_advanced_max',
 'Team_B_TRB%_advanced_totals',
 'Team_B_TRB_basic_max',
 'Team_B_TRB_basic_totals',
 'Team_B_TS%_advanced_max',
 'Team_B_TS%_advanced_totals',
 'Team_B_Total',
 'Team_B_USG%_advanced_max',
 'Team_B_USG%_advanced_totals',
 'Team_B_eFG%_advanced_max',
 'Team_B_eFG%_advanced_totals']



In [ ]:
df_explore = df[numerical_features]
features = df_explore.columns
n = len(features)

correlation_matrix = pd.DataFrame(np.zeros((n, n)), index=features, columns=features)
pval_matrix = pd.DataFrame(np.zeros((n, n)), index=features, columns=features)
for i in range(n):
    for j in range(n):
        if i == j:
            correlation_matrix.iloc[i, j] = 1.0
            pval_matrix.iloc[i, j] = 0.0
        else:
            r, p = pearsonr(df_explore.iloc[:, i], df_explore.iloc[:, j])
            correlation_matrix.iloc[i, j] = r
            pval_matrix.iloc[i, j] = p

flat_corrs = []
for i in range(n):
    for j in range(i+1, n):  
        r = correlation_matrix.iloc[i, j]
        p = pval_matrix.iloc[i, j]
        f1 = features[i]
        f2 = features[j]
        flat_corrs.append((f1, f2, r, p))

correlation_df = pd.DataFrame(flat_corrs, columns=["Feature 1", "Feature 2", "Correlation", "P-Value"])
correlation_df = correlation_df.sort_values(by="Correlation", key=abs, ascending=False)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_rows', None)
correlation_df.head(50)

In [ ]:
for col in numerical_features:
    correlation, p_value = sps.pointbiserialr(df[col], df['Team_A_win_next'])
    print(f'Correlation between {col} and Team_A_win_next: {correlation}, p: {p_value}')

The Point biserial correlation is calculated between each numerical features and the target label Team_A_win_next. Point biserial correlation measures the 

In [ ]:
df.shape

In [ ]:
#for col in numerical_features:
    #distance_correlation = dcor.distance_correlation(df[col].astype(float), df['Team_A_win_next'].astype(float))
    #print(f'dCor({col}, Team_A_win_next): {distance_correlation:.4f}')

In [ ]:
df_sample = df.groupby('Team_A_win_next', group_keys=False).apply(lambda x: x.sample(frac=5000 / len(df), random_state=42)).reset_index(drop=True)
for col in numerical_features:
    distance_correlation = dcor.distance_correlation(df[col].astype(float), df['Team_A_win_next'].astype(float))
    distance_cov = distance_covariance_test(df_sample[col].astype(float), df_sample['Team_A_win_next'].astype(float), num_resamples=500)
    p_val = distance_cov.pvalue
    print(f'dCor({col}, Team_A_win_next): {distance_correlation:.4f}, p = {p_val:.4f}')

In [ ]:
categorical_features =['Team_A_win','Team_A_home','playoff_game']
chi_square_results = {}
for col in categorical_features:
        contingency_table = pd.crosstab(df[col], df['Team_A_win_next']) # frequency dist of variables, row correponds to categorical variable col corresponds to target variable
        chi2, p, dof, expected = sps.chi2_contingency(contingency_table)
        chi_square_results[col] = (chi2, p)
        print(f'Feature: {col}, Chi square statistic: {chi2:.4f}, p = {p}')

In [ ]:
#df_sample = df.sample(n=3000, random_state=42)
#for col in numerical_features:
    #distance_cov = distance_covariance_test(df_sample[col].astype(float), df_sample['Team_A_win_next'].astype(float), num_resamples=500)
    #p_val = distance_cov.pvalue
    #print(f'{col}: p = {p_val}')

In [ ]:
df = df.drop(columns=['Team_A_USG%_advanced_totals', 'Team_B_USG%_advanced_totals'])
#Correlation and p values both nan, values of USG%_advanced_totals constant 

In [ ]:
df["Team_A"] = df["Team_A"].replace({'NJN': 'BRK', 'NOH': 'NOP', 'CHA': 'CHO'})
df["Team_B"] = df["Team_B"].replace({'NJN': 'BRK', 'NOH': 'NOP', 'CHA': 'CHO'})
df.head(5)
print(df['Team_A'].value_counts())
print(df['Team_B'].value_counts())

In [ ]:
teams = pd.unique(df[['Team_A', 'Team_B']].values.ravel())
team_to_id = {team: idx for idx, team in enumerate(teams)}
id_to_team = {idx: team for team, idx in team_to_id.items()}  
df['Team_A'] = df['Team_A'].map(team_to_id)
df['Team_B'] = df['Team_B'].map(team_to_id)
df.head(5)

In [ ]:
print(df['Team_A'].value_counts())
print(df['Team_B'].value_counts())
df.to_csv('nba_game_df.csv', index=False)

In [ ]:
x_sequence  = [] 
y_sequence = []
n = 5
grouped = df.groupby('matchup_id')

for idx, row in df.iterrows():
    matchup_id = row['matchup_id']
    game_date = row['date']
    past_games = grouped.get_group(matchup_id)
    past_games = past_games[past_games['date'] < game_date]
    if len(past_games) < n:
        continue
    last_n = past_games.tail(n)
    x_seq = last_n.drop(columns=['matchup_id', 'date', 'Team_A_win_next']).values
    x_sequence.append(x_seq)
    label = last_n.iloc[-1]['Team_A_win_next']
    y_sequence.append(label)

x = np.array(x_sequence)  
y = np.array(y_sequence)


In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
total_seq = x.shape[0]
games_per_seq = x.shape[1]
features_per_game = x.shape[1] 
print(f'Total sequences = {total_seq}, Games per sequence = {games_per_seq}, Features per game = {features_per_game}')
print(f'Total Team_A_win_next labels = {y.shape[0]}')

In [ ]:
train_size = int(0.7 * len(x))  
val_size = int(0.15 * len(x))   
test_size = len(x) - train_size - val_size  

x_train = x[:train_size]
y_train = y[:train_size]
x_val = x[train_size:train_size+val_size]
y_val = y[train_size:train_size+val_size]
x_test = x[train_size+val_size:] 
y_test = y[train_size+val_size:] 

x_train_flat = x_train.reshape(-1, x_train.shape[-1])  # shape: (total sequences * games/seq, num_features)
numerical_indices = [all_columns.index(col) for col in numerical_features]

mean = x_train_flat[:, numerical_indices].mean(axis=0)
std = x_train_flat[:, numerical_indices].std(axis=0)
x_train[:, :, numerical_indices] = (x_train[:, :, numerical_indices] - mean) / std
x_val[:, :, numerical_indices] = (x_val[:, :, numerical_indices] - mean) / std
x_test[:, :, numerical_indices] = (x_test[:, :, numerical_indices] - mean) / std

np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)
np.save('x_val.npy', x_val)
np.save('y_val.npy', y_val)
np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)

